In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.graphics.tsaplots import month_plot
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import ParameterGrid

In [14]:
# Load
df = pd.read_csv('./data/DHS_weekly.csv')
df.rename(columns={'Date': 'ds', 'Total Individuals in Shelter': 'y'}, inplace=True)

# parse date
df['ds'] = pd.to_datetime(df['ds'], format="%m/%d/%Y")

print(df.info())

# get holidays
holidayDates = df[(df['Easter'] == 1) | (df['Thanksgiving'] == 1) | (df['Christmas'] == 1)]['ds']
holidays = pd.DataFrame({
    'holiday':'generated', 'ds': holidayDates, 'lower window':-3, 'upper window':3
    })
holidays



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ds            366 non-null    datetime64[ns]
 1   y             366 non-null    int64         
 2   Easter        366 non-null    int64         
 3   Thanksgiving  366 non-null    int64         
 4   Christmas     366 non-null    int64         
 5   Temperature   366 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 17.3 KB
None


,holiday,ds,lower window,upper window
15,generated,2014-04-20,-3,3
47,generated,2014-11-30,-3,3
51,generated,2014-12-28,-3,3
65,generated,2015-04-05,-3,3
99,generated,2015-11-29,-3,3
103,generated,2015-12-27,-3,3
116,generated,2016-03-27,-3,3
151,generated,2016-11-27,-3,3
155,generated,2016-12-25,-3,3
171,generated,2017-04-16,-3,3


In [17]:
# Assuming the test set is intended to be for the last 60 days in the dataset
max_date = df['ds'].max()
split_date = max_date - pd.Timedelta(weeks=13)

# Split the data into training and testing sets based on the split date
train_df = df[df['ds'] <= split_date]
test_df = df[df['ds'] > split_date]


In [18]:
from prophet import Prophet

m = Prophet(
    holidays=holidays,
)
m.add_regressor('Temperature')
m.fit(train_df)

20:15:20 - cmdstanpy - INFO - Chain [1] start processing
20:15:20 - cmdstanpy - INFO - Chain [1] done processing


In [19]:
# Regressor Coefficients
from prophet.utilities import regressor_coefficients
coef = regressor_coefficients(m)

# Function to interpret the coefficient results
def interpret_prophet_coefficients(df):
    interpretations = []

    # Iterate through each row in the DataFrame 'df'
    for _, row in df.iterrows():
        regressor = row['regressor']  # Get the regressor name
        mode = row['regressor_mode']  # Get the regressor mode (multiplicative or additive)
        coef = row['coef']  # Get the coefficient value
        effect_type = 'increase' if coef > 0 else 'decrease'  # Determine if the effect is an increase or decrease

        # Generate interpretation based on the regressor mode
        if mode == 'multiplicative':
            interpretation = f"For each unit increase in {regressor}, the target variable is expected to {effect_type} by {abs(coef) * 100:.2f}% (multiplicatively)."
        elif mode == 'additive':
            interpretation = f"For each unit increase in {regressor}, the target variable changes by {coef:.2f} units (additively)."
        else:
            interpretation = f"Regressor {regressor} has an unrecognized mode '{mode}'."

        interpretations.append(interpretation)

    return interpretations

coefs = pd.DataFrame(coef)  # Get regressor coefficients from the Prophet model
interpretations = interpret_prophet_coefficients(coefs) # Generate interpretations based on coefficients

# Print each interpretation
for interpretation in interpretations:
    print(interpretation)

For each unit increase in Temperature, the target variable changes by -10.78 units (additively).


In [ ]:
# Create a dataframe for predictions
future_df = m.make_future_dataframe(periods=13, freq='W')  # Generate future dates for 13 weeks
# print(future_df.tail())
# print(len(future_df))
# Include the regressors in the future dataframe
future_df = future_df.merge(df[['ds', 'Temperature']], on='ds', how='left')
# print(future_df.tail())
# print(len(future_df))
# Predict over the future dataframe
forecast = m.predict(future_df)
forecast

            ds
361 2020-12-06
362 2020-12-13
363 2020-12-20
364 2020-12-27
365 2021-01-03
366
            ds  Temperature
361 2020-12-06    10.072857
362 2020-12-13     8.208571
363 2020-12-20     3.535714
364 2020-12-27     7.510000
365 2021-01-03     6.625000
366


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Temperature,Temperature_lower,Temperature_upper,additive_terms,...,holidays,holidays_lower,holidays_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2014-01-05,351097.710120,353232.257716,359822.333621,351097.710120,351097.710120,151.828475,151.828475,151.828475,5517.963258,...,0.000000,0.000000,0.000000,5366.134783,5366.134783,5366.134783,0.0,0.0,0.0,356615.673378
1,2014-01-12,352468.509764,355116.924020,361795.200609,352468.509764,352468.509764,125.858211,125.858211,125.858211,6057.192884,...,0.000000,0.000000,0.000000,5931.334673,5931.334673,5931.334673,0.0,0.0,0.0,358525.702648
2,2014-01-19,353839.309407,357550.444429,364283.585776,353839.309407,353839.309407,100.248303,100.248303,100.248303,7105.355297,...,0.000000,0.000000,0.000000,7005.106994,7005.106994,7005.106994,0.0,0.0,0.0,360944.664704
3,2014-01-26,355210.109051,359739.085289,366500.776332,355210.109051,355210.109051,180.912584,180.912584,180.912584,7856.667731,...,0.000000,0.000000,0.000000,7675.755147,7675.755147,7675.755147,0.0,0.0,0.0,363066.776781
4,2014-02-02,356580.908694,361015.901543,367283.562671,356580.908694,356580.908694,148.280356,148.280356,148.280356,7571.226632,...,0.000000,0.000000,0.000000,7422.946276,7422.946276,7422.946276,0.0,0.0,0.0,364152.135326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2020-12-06,387146.174594,392236.547049,398992.888223,386635.866351,387654.264310,52.385649,52.385649,52.385649,8408.506685,...,0.000000,0.000000,0.000000,8356.121036,8356.121036,8356.121036,0.0,0.0,0.0,395554.681279
362,2020-12-13,386726.932208,391550.431847,398498.444394,386092.144658,387335.993536,72.482419,72.482419,72.482419,8277.132499,...,0.000000,0.000000,0.000000,8204.650080,8204.650080,8204.650080,0.0,0.0,0.0,395004.064707
363,2020-12-20,386307.689823,390143.976381,397110.138165,385536.107976,387070.912658,122.855245,122.855245,122.855245,7418.641219,...,0.000000,0.000000,0.000000,7295.785973,7295.785973,7295.785973,0.0,0.0,0.0,393726.331042
364,2020-12-27,385888.447438,388008.431229,394808.947839,384971.560762,386797.902302,80.012934,80.012934,80.012934,5542.744064,...,-616.405443,-616.405443,-616.405443,6079.136574,6079.136574,6079.136574,0.0,0.0,0.0,391431.191502


In [24]:
from prophet.plot import plot_plotly
plot_plotly(m,forecast.reset_index())

## Find better parameters

In [27]:
param_grid = {
    'changepoint_prior_scale': [0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.1, 1.0, 10.0],
    'holidays_prior_scale': [0.1, 1.0, 10.0],
    'seasonality_mode': ['additive', 'multiplicative']
}

grid = ParameterGrid(param_grid)
all_params = list(grid)
total_params = len(all_params)
print(f'all params: {total_params}')



all params: 54


In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics
def train_model(params):
    m = Prophet(
        seasonality_mode=params['seasonality_mode'],
        holidays=holidays,
        seasonality_prior_scale=params['seasonality_prior_scale'],
        holidays_prior_scale=params['holidays_prior_scale'],
        changepoint_prior_scale=params['changepoint_prior_scale']
    )
    m.add_regressor('Temperature')
    m.fit(df)
    return m

best_rmse = float('inf')
best_params = None

results = []

for index, params in enumerate(all_params):
    print(f'({index+1}/{total_params})testing params: {params}')
    m = train_model(params)
    df_cv = cross_validation(model=m, period='42 days', initial='1500 days', horizon='91 days', parallel="processes")
    performance = performance_metrics(df_cv)
    mean_rmse = performance['rmse'].mean()
    results.append(mean_rmse)
    if mean_rmse < best_rmse:
        best_rmse = mean_rmse
        best_params = params
    print(f'params: {params}, rmse: {mean_rmse}')

In [31]:
outcome = pd.DataFrame(all_params)
outcome['tuning_results'] = results
print(outcome.loc[outcome['tuning_results'].idxmin()])

changepoint_prior_scale               0.1
holidays_prior_scale                 10.0
seasonality_mode           multiplicative
seasonality_prior_scale               0.1
tuning_results                9635.596564
Name: 33, dtype: object


In [33]:
m = train_model(best_params)
forecast = m.predict(future_df)
plot_plotly(m,forecast.reset_index())

20:35:05 - cmdstanpy - INFO - Chain [1] start processing
20:35:05 - cmdstanpy - INFO - Chain [1] done processing
